Предполагается, что в рамках данного задания ученики сами будут изучать документацию, как минимум используя функционал `tab` и `shift+tab` от Jupyter

1. Импортируйте библиотеку pandas 
2. Считайте файл `wage.csv` в DataFrame с названием `wage`. Используйте для этого функцию pd.read_csv()

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

In [195]:
import pandas as pd
df = pd.read_csv('wage.csv', encoding='utf-8')

3. Поменяйте колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

In [196]:
def replace_FM(anItem):
    if anItem == 0:
        return 'F' 
    elif anItem == 1:
        return 'M'


df.gender = df.gender.apply(replace_FM)

4. Давайте посчитаем среднюю зарплату у мужчин и женщин. Для этого используйте метод `groupby` со следующим синтаксисом:

`dataframe.groupby(Название_колонки_для_группировки)[Перечисление_колонок_для_агрегации].функция_агрегации()`

In [201]:
# 1-й вариант
amount_M = df.groupby(df.gender)['person_id'].count().M
amount_F = df.groupby(df.gender)['person_id'].count().F
avg_salary_M = sum(df[df.gender == 'M'].wage.values) / amount_M
avg_salary_F = sum(df[df.gender == 'F'].wage.values) / amount_F

In [202]:
# 2-й вариант
avg_salary_M = df[df.gender == 'M'].wage.values.mean()
avg_salary_F = df[df.gender == 'F'].wage.values.mean()

In [203]:
# 3-й вариант
df.groupby(df.gender)['wage'].mean()

gender
F    40855.747261
M    46815.944005
Name: wage, dtype: float64

5. Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз. 
    1. Найдите таких людей. Подсказка: `value_counts()`
    0. Убедитесь, что записи по ним с одинаковым `wage`. Возможно, тут вам пригодится функция агрегации `nunique()`, отображающая количество разных значений
    0. Избавьтесь от повторяющихся значений. Подсказка: `drop_duplicates()`

In [204]:
# A
df.value_counts()[df.value_counts() > 1]

person_id  gender  wage         
17         M       20178.985060     2
16         F       35448.040878     2
15         F       68341.332163     2
14         M       13434.655493     2
13         M       273483.609236    2
12         M       23547.663281     2
dtype: int64

In [207]:
#B
df[df.person_id == 12]

In [208]:
df = df.drop_duplicates()

6. Теперь посмотрим внимательнее на зарплаты
    1. Охарактеризуйте имеющиеся данные по зарплатам. Подсказка: `describe`
    1. Избавьтесь от бессмысленных значений

In [210]:
df.wage.describe();

In [212]:
df = df.drop(labels = df.wage[df.wage <= 0].index,axis = 0)
df.describe()

7. Давайте теперь посмотрим на зарплату с учетом бонуса. Для этого нам понадобится таблица `bonus.csv`. Считайте ее в переменную `bonus`. Заметьте, что она сохранена немного в другом формате, и вам понадобится уточнить параметр `sep` - разделитель записей. Сравните текущий файл с предыдущим и попробуйте решить проблему

In [214]:
df_bonus = pd.read_csv('bonus.csv', sep=';' ,encoding='utf-8')
df_bonus

8. Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет соединить (сджойнить) таблицы по `person_id`. Используйте для этого функцию `pd.merge`. Помните, что параметр `how` должен быть `'outer'`, чтобы сохранить те записи, что есть только в одной таблице. Результат запишите в новый dataframe `df`

In [215]:
merged_df = df.merge(df_bonus, how='outer', on='person_id')
merged_df

,person_id,gender,wage,bonus
0,0,M,46793.603811,3.332934e+04
1,1,M,33481.575720,NaN
2,2,M,44523.699084,3.192912e+06
3,3,M,15995.576829,2.196858e+04
4,4,F,10282.631224,NaN
...,...,...,...,...
994,997,F,104504.616392,5.380978e+04
995,998,M,98927.903076,NaN
996,999,M,108107.141368,NaN
997,43,NaN,NaN,2.687784e+05


9. Наконец, давайте посчитаем итоговую зарплату
    1. Замените отсутствующие записи в колонке `bonus` нулями
    1. Уберите людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
    1. Сделайте новую колонку `total`, которая будет равна 12 окладам и премии
    1. Посчитайте среднюю и медианную итоговую зарплату в разрезе по полу. Подсказка: вместо функции агрегации можно написать `.agg()` и перечислить внутри нужные агрегаты

In [216]:
#A
merged_df = merged_df.fillna(0)
merged_df

,person_id,gender,wage,bonus
0,0,M,46793.603811,3.332934e+04
1,1,M,33481.575720,0.000000e+00
2,2,M,44523.699084,3.192912e+06
3,3,M,15995.576829,2.196858e+04
4,4,F,10282.631224,0.000000e+00
...,...,...,...,...
994,997,F,104504.616392,5.380978e+04
995,998,M,98927.903076,0.000000e+00
996,999,M,108107.141368,0.000000e+00
997,43,0,0.000000,2.687784e+05


In [217]:
#B
merged_df = merged_df[merged_df.wage != 0]
merged_df

,person_id,gender,wage,bonus
0,0,M,46793.603811,3.332934e+04
1,1,M,33481.575720,0.000000e+00
2,2,M,44523.699084,3.192912e+06
3,3,M,15995.576829,2.196858e+04
4,4,F,10282.631224,0.000000e+00
...,...,...,...,...
992,995,M,66503.737185,3.452137e+03
993,996,M,9972.956272,3.892790e+05
994,997,F,104504.616392,5.380978e+04
995,998,M,98927.903076,0.000000e+00


In [219]:
#C
merged_df['total'] = merged_df.wage * 12 + merged_df.bonus
merged_df['total']

C:\Users\lipton\AppData\Local\Temp\ipykernel_63308\1859682464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['total'] = merged_df.wage * 12 + merged_df.bonus


0      5.948526e+05
1      4.017789e+05
2      3.727197e+06
3      2.139155e+05
4      1.233916e+05
           ...     
992    8.014970e+05
993    5.089544e+05
994    1.307865e+06
995    1.187135e+06
996    1.297286e+06
Name: total, Length: 997, dtype: float64

In [221]:
#D
# 1-й вариант
merged_df.total.agg(['mean', 'median'])

mean      614507.580935
median    384201.320027
Name: total, dtype: float64

In [223]:
# 2-й вариант
merged_df.groupby(merged_df.gender)['total'].agg(['mean', 'median'])

,mean,median
gender,,
F,570746.139432,347622.913892
M,657142.490282,437499.824868


In [224]:
merged_df

,person_id,gender,wage,bonus,total
0,0,M,46793.603811,3.332934e+04,5.948526e+05
1,1,M,33481.575720,0.000000e+00,4.017789e+05
2,2,M,44523.699084,3.192912e+06,3.727197e+06
3,3,M,15995.576829,2.196858e+04,2.139155e+05
4,4,F,10282.631224,0.000000e+00,1.233916e+05
...,...,...,...,...,...
992,995,M,66503.737185,3.452137e+03,8.014970e+05
993,996,M,9972.956272,3.892790e+05,5.089544e+05
994,997,F,104504.616392,5.380978e+04,1.307865e+06
995,998,M,98927.903076,0.000000e+00,1.187135e+06


10. Сохраните `df` в файл, используя метод `to_csv()`. Не записывайте индексы

In [225]:
import csv
with open('merged_df.csv', 'w', newline='') as csv_file:
    fieldnames = ['gender', 'wage', 'bonus', 'total']
    writer = csv.writer(csv_file)
    writer.writerow(
        fieldnames
    )
def write_csv(anItem):
    with open('merged_df.csv', 'a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        for i in range(len(anItem)):
            writer.writerow(anItem[fieldnames].loc[i])
write_csv(merged_df)